# Motion generation from checkpoints

In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import FoundationXML
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints
import PythonKit

In [ ]:
let np  = Python.import("numpy")

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

## Set training params

In [ ]:
let device = Device.defaultTFEager

In [ ]:
let maxTextSequenceLength =  20
let maxMotionLength =  100

In [ ]:
let datasetSize: DatasetSize = .full
let batchSize = 150

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor(vocabulary: vocabulary, tokenizer: tokenizer)

// model config
let config = LangMotionTransformerConfig(
    vocabSize: vocabulary.count,
    nbJoints: 47, // TODO: get value from dataset
    nbMixtures: 20,
    layerCount: 6,
    modelSize: 256,
    feedForwardSize: 1024,
    headCount: 8,
    dropoutProbability: 0.1,
    sentenceMaxPositionalLength: 100, 
    motionMaxPositionalLength: 500,
    doMotionDense: false
)

In [ ]:
print("\nLoading dataset...")

var dataset = try Lang2Motion(
    motionDatasetURL: motionDatasetURL,
    batchSize: batchSize,
    minMotionLength: 10,
    maxMotionLength: 100,
    trainTestSplit: 1.0,
    device: device
) { (motionSample: MotionSample) -> LangMotionBatch in    
    let sentence = textProcessor.preprocess(sentence: motionSample.annotations[0], maxTextSequenceLength: maxTextSequenceLength)
    let (motionPart, target) = LangMotionBatch.preprocessTargetMotion(sampleID: motionSample.sampleID, motion: motionSample.motion, maxMotionLength: maxMotionLength)
    let source = LangMotionBatch.Source(sentence: sentence, motionPart: motionPart)
    let singleBatch = LangMotionBatch(data: source,label: target)
    return singleBatch
}

print("Dataset acquired.")

## Helpers

In [ ]:
public class MotionDecoder2 {
    public static func greedyDecodeMotion(
        sentence: LangMotionBatch.Sentence, 
        startMotion: Tensor<Float>?,
        transformer: LangMotionTransformer, 
        nbJoints: Int, 
        nbMixtures: Int, 
        maxMotionLength: Int,
        memoryMultiplier: Float = 1.0
    ) -> Tensor<Float> {
        print("\nEncode:")
        print("======")
        let memory = transformer.encode(input: sentence) //* memoryMultiplier
        print("  memory.count: \(memory.shape)")     

        print("\nGenerate:")
        print("=========")

        // start with tensor for neutral motion frame
        let zeroMotionFrame = LangMotionBatch.zeroMotionFrame(nbJoints: nbJoints).expandingShape(at: 0)
        var ys: Tensor<Float> = zeroMotionFrame
        // or with supplied motion
//         if startMotion != nil {
//             ys = startMotion!.expandingShape(at:0)            
//         }

        print("ys.shape: \(ys.shape)")
        
        let maxMotionLength2 = maxMotionLength-ys.shape[1]+1
//         print("maxMotionLength2: \(maxMotionLength2)")
        
        for step in 0..<maxMotionLength2 {
            // print("step: \(step)")
            // prepare input
            let motionPartFlag = Tensor<Int32>(repeating: 1, shape: [1, ys.shape[1]])
            let motionPartMask = LangMotionBatch.makeStandardMask(target: motionPartFlag, pad: 0)
            let previous_ys = Tensor<Float>(concatenating: [zeroMotionFrame, ys], alongAxis: 1)[0..., 0...ys.shape[1]-1, 0...]
            var motionStartFlag = Tensor<Float>(zeros: [ys.shape[1], 1]).expandingShape(at: 0) // FIXME: refactor getting motionStartFlag
            motionStartFlag[0, 0, 0] = Tensor(1.0)
            let motionPart = LangMotionBatch.MotionPart(motion: ys, mask: motionPartMask, previousMotion: previous_ys, startFlag: motionStartFlag)
            
            
            // decode motion
            let dedoderOutput = transformer.decode(sourceMask: sentence.mask, motionPart: motionPart, memory: memory)
            
//             print("dedoderOutput")
            
            let mixtureModelInput = Tensor<Float>(concatenating: dedoderOutput.allOutputs, alongAxis: 2)
//             print("mixtureModelInput.shape: \(mixtureModelInput.shape)")
            let singlePreds = transformer.mixtureModel(mixtureModelInput[0...,-1].expandingShape(at: 0))

            
            // perform sampling
            let (sampledMotion, _, _) = MotionDecoder.performNormalMixtureSampling(
                preds: singlePreds, nb_joints: nbJoints, nb_mixtures: nbMixtures, maxMotionLength: maxMotionLength)
            
            // concatenate motion
            ys = Tensor(concatenating: [ys, sampledMotion.expandingShape(at: 0)], alongAxis: 1)        
        }
        return ys.squeezingShape(at:0)[1...]
    }    
}

In [ ]:
public struct SampleMotionClip {
    var sampleID: Int
    var start: Int = 0
    var length: Int = 1
}

public func getClippedMotionFrames(dataset: Lang2Motion, clipInfo: SampleMotionClip?) -> Tensor<Float>? {
    if clipInfo != nil {
    
    let ms: MotionSample = dataset.motionSamples.filter { $0.sampleID == clipInfo!.sampleID } [0]
    let clippedMotionFrames = ms.motion[clipInfo!.start..<clipInfo!.start+clipInfo!.length]
    return clippedMotionFrames
    } else {
        return nil
    }
}

In [ ]:
public func greedyDecodeMotion(dataset: Lang2Motion, model: LangMotionTransformer, sentence: String, leadingFrames: SampleMotionClip?, prefix: String = "prefix", saveMotion: Bool = true, memoryMultiplier: Float = 0.0, motionsURL: URL?) {
    let startMotion: Tensor<Float>? = getClippedMotionFrames(dataset: dataset, clipInfo: leadingFrames)
    var leadingFramesStr = "0"
    if startMotion != nil {
        leadingFramesStr = "\(startMotion!.shape[0])"
    }
    // TODO: incorporate done/stop signal
    Context.local.learningPhase = .inference
    print("\ngreedyDecodeMotion(sentence: \"\(sentence)\")")

    let processedSentence = textProcessor.preprocess(sentence: sentence, maxTextSequenceLength: maxTextSequenceLength)
    processedSentence.printSentence()

    let decodedMotion = MotionDecoder2.greedyDecodeMotion(
        sentence: processedSentence, 
        startMotion: startMotion,
        transformer: model, 
        nbJoints: config.nbJoints, 
        nbMixtures: config.nbMixtures, 
        maxMotionLength: maxMotionLength,
        memoryMultiplier: memoryMultiplier
    )
    print("  decodedMotion: min: \(decodedMotion.min()), max: \(decodedMotion.max())")
    let descaledMotion = dataset.scaler.inverse_transform(decodedMotion)
    print("  descaledMotion.shape: \(descaledMotion.shape)")
    print("  descaledMotion: min: \(descaledMotion.min()), max: \(descaledMotion.max())")
    var imageURL: URL? = nil
    
    if !saveMotion { imageURL = nil } else {
        imageURL = motionsURL!.appendingPathComponent("\(prefix).png")
    }
    // use joint groupping
    let grouppedJointsMotion = MotionSample.grouppedJoints(motion: descaledMotion, jointNames: dataset.motionSamples[0].jointNames)
    motionToImg(url: imageURL, motion: grouppedJointsMotion, motionFlag: nil, padTo: maxMotionLength, descr: "\(sentence), LF: \(leadingFramesStr)", cmapRange: 1.0)

    if saveMotion {
        print("Saved image: \(imageURL!.path)")
        let jointNames = dataset.motionSamples[0].jointNames
        let mmmXMLDoc = MMMWriter.getMMMXMLDoc(jointNames: jointNames, motion: descaledMotion)
        let mmmURL = motionsURL!.appendingPathComponent("\(prefix).mmm.xml")
        try! mmmXMLDoc.xmlData(options: XMLNode.Options.nodePrettyPrint).write(to: mmmURL)
        print("Saved motion: \(mmmURL.path)")
    }
}

In [ ]:
func showMotionSample(_ motionSample: MotionSample) {
    let motion = motionSample.motion
    let descaledMotion = dataset.scaler.inverse_transform(motion)
    let sentence = "sample_id=\(motionSample.sampleID), ann=\(motionSample.annotations[0])"

    print("motion: min: \(motion.min()), max: \(motion.max())")
    print("descaledMotion.shape: \(descaledMotion.shape)")
    print("descaledMotion: min: \(descaledMotion.min()), max: \(descaledMotion.max())")

    // use joint groupping
    let jointNames = dataset.motionSamples[0].jointNames
    let grouppedJointsMotion = MotionSample.grouppedJoints(motion: descaledMotion, jointNames: dataset.motionSamples[0].jointNames)
    motionToImg(url: nil, motion: grouppedJointsMotion, motionFlag: nil, padTo: maxMotionLength, descr: sentence, cmapRange: 1.0)
}

In [ ]:
func showMotion(motion: Tensor<Float>) {
    let descaledMotion = dataset.scaler.inverse_transform(motion)
    let grouppedJointsMotion = MotionSample.grouppedJoints(motion: descaledMotion, jointNames: dataset.motionSamples[0].jointNames)
    motionToImg(url: nil, motion: grouppedJointsMotion, motionFlag: nil, padTo: maxMotionLength, descr: "", cmapRange: 1.0)
}

In [ ]:
func saveMotionToMMM(motion: Tensor<Float>, mmmURL: URL) {
    let descaledMotion = dataset.scaler.inverse_transform(motion)
    let jointNames = dataset.motionSamples[0].jointNames
    let mmmXMLDoc = MMMWriter.getMMMXMLDoc(jointNames: jointNames, motion: descaledMotion)
    try! mmmXMLDoc.xmlData(options: XMLNode.Options.nodePrettyPrint).write(to: mmmURL)
    print("Saved motion: \(mmmURL.path)")
}

## Load model checkpoint

## Decode using leading motion frames

## Load model checkpoint

In [ ]:
let runName = "run_37"
let epoch = 16

In [ ]:
let runURL = dataURL.appendingPathComponent("runs/Lang2motion/\(runName)", isDirectory: true)
let checkpointURL = runURL.appendingPathComponent("checkpoints", isDirectory: true)
let motionsURL = runURL.appendingPathComponent("generated_motions", isDirectory: true)
try! FileManager().createDirectory(at: motionsURL, withIntermediateDirectories: true)

let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.e\(epoch)")
// let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.final")

### Generate motion

In [ ]:
var genNum = 1

In [ ]:
var s: String = ""
var lf: SampleMotionClip?

In [ ]:
// s = "A person is walking forwards five steps."
// s = "A person is walking forwards."
// lf = SampleMotionClip(sampleID: 1, start: 26, length: 2)
lf = nil

// s = "A person plays the guitar."
// lf = SampleMotionClip(sampleID: 1417, start: 14, length: 10)

// s = "The human plays air guitar and sways ans stands still."
// s = "The human walks in the straight line."
// s = "Someone is jogging."

// s = "a person waves with his both arms"
// s = "a person is waving his hand."
s = "A person runs."
// s = "A person kneels down."
// s = "A human walking backwards"
// s = "A person walks 4 steps forward."

In [ ]:
greedyDecodeMotion(dataset: dataset, model: model, sentence: s, leadingFrames: lf, 
    prefix: "epoch_\(epoch)_motion_\(genNum)", 
    saveMotion: true,  memoryMultiplier: 1.0, motionsURL: motionsURL
)
genNum += 1